In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)

from analysis.lib.fitting import common, fit; reload(common); reload(fit)

import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline
def savefig(name):
    plt.savefig(name + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(name + ".pdf", bbox_inches='tight')

In [ ]:
reload(pu_delayfb)

det6_ts = 20170531141332
det12_ts  = 20170531142610

det6_x, det6_y, det6_y_u, det6_fr = pu_delayfb.calibrate_LDE_phase(
    contains="phase_fb_delayline", older_than=str(det6_ts + 1), do_fit=True, ret_data_fit=True,
    fixed=[0,1]
)

det12_x, det12_y, det12_y_u, det12_fr = pu_delayfb.calibrate_LDE_phase(
    contains="phase_fb_delayline", older_than=str(det12_ts + 1), do_fit=True, ret_data_fit=True,
    fixed=[0,1]
)

In [ ]:
plt.figure()

eb6 = plt.errorbar(det6_x, det6_y, det6_y_u, fmt='o', label=r'$6\degree$ over-rotation')
eb12 = plt.errorbar(det12_x, det12_y, det12_y_u, fmt='o', label=r'$12\degree$ over-rotation')

def avg_param(param):
    return (det6_fr['params_dict'][param] + det12_fr['params_dict'][param]) / 2.0

avg_T = avg_param('t')
avg_phi = avg_param('phi')
avg_A = avg_param('A')

_, ff_6, _ = common.fit_decaying_cos(6./360., 0.0, avg_A, avg_phi, avg_T)
_, ff_12, _ = common.fit_decaying_cos(12./360., 0.0, avg_A, avg_phi, avg_T)

plot_xvals = np.linspace(0,60,1000)

plt.plot(plot_xvals, ff_6(plot_xvals), color=eb6[0].get_color())
plt.plot(plot_xvals, ff_12(plot_xvals), color=eb12[0].get_color())

plt.xlim(0, 60)
plt.ylim(-1, 1)
plt.legend(loc=1)
plt.xlabel("number of LDE attempts")
plt.ylabel(r"$\langle X \rangle$")
savefig("thesis-plots/phase-control-detuning")

In [ ]:
avg_phi

In [ ]:
2/360.0/443e3

In [ ]:
carbons = np.arange(1,8)

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)
import itertools

carbon_combis = list(itertools.combinations(carbons, 2))
import scipy.stats

freqs = np.zeros((len(carbons) + 2*len(carbon_combis), 3)) # -1, 0, +1
T2stars = np.zeros((len(carbons) + 2*len(carbon_combis)))

c_idxs = dict()

T2_star_us = np.array([
    [0.14, 0.15],
    [0.26, 0.15],
    [0.09, 0.11],
    [0.48, 0.33],
    [0.05, 0.06],
    [0.12, 0.10],
    [0.08, 0.10]
])

T2_star_u = np.sqrt(np.sum(T2_star_us**2, axis=1))

for i_c, c in enumerate(carbons):
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    msp1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "p1")]
    msm1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "m1")]
    T2star_0 = msmt_params.cfg['samples']['111no2']['C%d_T2star_0' % (c)]
    T2star_1 = msmt_params.cfg['samples']['111no2']['C%d_T2star_1_%s' % (c, "m1")]
    
    T2star_arr = np.array([T2star_0, T2star_1])
    # take the sqrt of the harmonic mean of the squared T2stars
    T2star = np.sqrt(scipy.stats.hmean(T2star_arr ** 2))
    print("avg. T2* for C%d: %.2f" % (c, T2star * 1e3))
    T2stars[i_c] = T2star
    
    
    freqs[i_c,0] = msm1_freq
    freqs[i_c,1] = ms0_freq
    freqs[i_c,2] = msp1_freq
    
    print("C%d dephasing rate parameter: %.3f kHz" % (c, 1e-3 * (msp1_freq-msm1_freq)) )
#     T2stars[i_c] = T2star
    
    c_idxs[str(c)] = i_c
    
# 
# +1 frequencies of C4 and C5 are interchanged
# freqs[3:5,2] = freqs[3:5,2][::-1]
    
for i_cc, cc in enumerate(carbon_combis):
    idx = 2*i_cc + len(carbons)
    freqs[idx,:] = (freqs[c_idxs[str(cc[0])],:] + freqs[c_idxs[str(cc[1])],:])
    print("C%s dephasing rate parameter: %.3f kHz" % ("%d%d+" % cc, 1e-3 * (freqs[idx,2] - freqs[idx,0])) )
    c_idxs["%d%d+" % cc] = idx
    freqs[idx+1,:] = (freqs[c_idxs[str(cc[0])],:] - freqs[c_idxs[str(cc[1])],:])
    print("C%s dephasing rate parameter: %.3f kHz" % ("%d%d-" % cc, 1e-3 * (freqs[idx+1,2] - freqs[idx+1,0])) )
    c_idxs["%d%d-" % cc] = idx+1
    

freqs[3,2], freqs[4,2] = freqs[4,2], freqs[3,2]
freqs = freqs[0:7,:]
    
print freqs

In [ ]:
freqs[0:7,:]

In [ ]:
np.mean(freqs[0:7,1])

In [ ]:
np.round((freqs[0:7,2] - freqs[0:7,1]) / 1e2)

In [ ]:
np.round(freqs[0:7,1] / 1e2)

In [ ]:
A_par = (freqs[:,2]**2 - freqs[:,0]**2)/(4*freqs[:,1])
A_perp = np.sqrt((freqs[:,2]**2 + freqs[:,0]**2)/2 - freqs[:,1]**2 - A_par**2)

In [ ]:
np.round(A_perp / 1e2)

In [ ]:
T2_star_u

In [ ]:
T2stars[0:7]*1e3

In [ ]:
np.round(np.array([443349.78,  442982.61,  442978.96, 442824.75, 443732.93, 443851.31, 443261.6])/1e2)
